In [23]:
for QK, N_O in QorK.items():
    
    for an_postition, acc_num in enumerate(N_O):
        year = int(acc_num.split(sep='-')[1])
        print(f'year - {year}')
        pages = master_filings_dict[acc_num]['filing_documents'][QK]['pages_code']
        print('\n', 'This is {} a {} document.'.format(acc_num, QK))
        print(len(pages))
        
        for page in pages: 
            
            if anchor_check(page) and search(r'million', page, flags=re.I):
                print('anchor check')
                # Convert string to BS object
                soup = BeautifulSoup(page, 'html5')

                # Get only the first table in the page.
                for table_num, table in enumerate(soup.find_all('tbody')):
                    
                    if table_num == 0:
                        
                        # then get all the rows in the table.
                        table_rows = soup.findAll('tr')
                       
                        if len( [tr.text for tr in table_rows[0].find_all('td')] ) < 3:
                            del table_rows[0]

                        single_table = []
                        Date_column = []

                        # Loop through each column, adding in three structures to single_table.
                        for tr_post, tr in enumerate(table_rows):
                            td = tr.find_all('td')

                            try: 
                                column = [tr.text.encode('cp1252').decode("utf-8", "ignore") for tr in td]
                                
                            except: 
                                column  =[tr.text.encode('ISO-8859-1').decode("utf-8", "ignore") for tr in td]
                                
#                             print( 'tr_post', tr_post, len(column), column)

                            if QK == '10-Q' and year < 11 or QK == '10-K' and year < 13:
                                
                                if tr_post == 2:

                                    # Quarter Column
                                    col_Q, col_year = Quarter( column, '10-Q' )
                                    single_table.append( col_Q )
                                    # Year Column
                                    single_table.append( col_year )

                                elif tr_post > 1: 
#                                     print(column)
                                    # Features Columns
                                    cleaned_data = cleaning_column( column , 10**6 )
                                    #print(cleaned_data)
                                    if len(cleaned_data) > 10:
                                        del cleaned_data[3]

                                    if cleaned_data == ['']:
                                        break
                                    single_table.append( cleaned_data )
                            else: 

                                if tr_post == 1:

                                    # Quarter Column
                                    col_Q, col_year = Quarter( column, '10-Q' )
                                    single_table.append( col_Q )
                                    # Year Column
                                    single_table.append( col_year )

                                elif tr_post > 1: 
            #                         print(column)
                                    # Features Columns
                                    cleaned_data = cleaning_column( column , 10**6 )
                                    #print(cleaned_data)
                                    if len(cleaned_data) > 10:
                                        del cleaned_data[3]

                                    if cleaned_data == ['']:
                                        break
                                    single_table.append( cleaned_data )
                                    
                        print(single_table[1])
                        table_df = pd.DataFrame(single_table)
        #                 print(table_df)
                        table_df = table_df.transpose()
        #                 print(table_df[0])

                table_df.drop(table_df.index[pick_row_del(single_table):], 0, inplace=True)

                table_df.columns = table_df.iloc[0]

                table_df = table_df.drop(table_df.index[0])
                ICD.display(table_df)
            
        else:
            print( '\n', '~'*40, '\n')
            

year - 20


KeyError: '0001564590-20-019706'

# Problems

###  - Apple doesnt have key in Table of Contents
### - Amazon has werid keys

# Capstone Project

# Libraries

In [2]:
# Import Library 

import re
import requests
from re import search

import unicodedata
from bs4 import BeautifulSoup
from bs4 import Tag

from bs4 import NavigableString
from bs4 import SoupStrainer
from pprint import pprint

import numpy as np
from numpy import array
import pandas as pd 

import csv
import pickle

from collections import Counter
import datetime

from IPython.display import display_html
from IPython.display import display
from IPython.core import display as ICD
pd.set_option('display.max_columns', 100)


# Initialise dictionary-containers and url templates. 

In [1]:
master_filings_dict = {}
        
Accession_Number_URL = {}

AN_Active = {}

name_CIK = { 
    'Tesla' :  ['0001318605']
}

dir_url = r'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={}&type=10-&dateb=&owner=include&count=100'

doc_url = r'https://www.sec.gov/Archives/edgar/data/{CIKx}/{xx}/{yy}.txt'

# Retrieving unique Accession Numbers per filing based on CIK.

In [3]:
# Dict of Companies -> CIK
for name, CIK_list in name_CIK.items():
    
    for CIK_position, CIK in enumerate(CIK_list):
        
        # CIK -> A_N 
        new_dir_url = dir_url.format( CIK )

        new_doc_url = doc_url.format(CIKx = CIK, xx='{xx}', yy='{yy}' )

        response = requests.get( new_dir_url )
        soup = BeautifulSoup( response.content, 'lxml' )
        text = soup.get_text( strip=True )

        cleaned_text = re.findall( 'Acc-no: \d+-\d+-\d+' , text )

        accession_number = [ n.replace('Acc-no: ', '') for n in cleaned_text ]
        accessionnumber = [ num.replace('-', '') for num in accession_number ]
        
        accession_numbers = zip(accessionnumber, accession_number)

        for a,b in accession_numbers: 
            
            file_url_list = new_doc_url.format(xx = a, yy = b)

            Accession_Number_URL.update({ b : file_url_list})
            


In [4]:
print('Accession_Number_URL - ', '\n', Accession_Number_URL)

Accession_Number_URL -  
 {'0001564590-20-019931': 'https://www.sec.gov/Archives/edgar/data/0001318605/000156459020019931/0001564590-20-019931.txt', '0001564590-20-018984': 'https://www.sec.gov/Archives/edgar/data/0001318605/000156459020018984/0001564590-20-018984.txt', '0001564590-20-004475': 'https://www.sec.gov/Archives/edgar/data/0001318605/000156459020004475/0001564590-20-004475.txt', '0001564590-19-038256': 'https://www.sec.gov/Archives/edgar/data/0001318605/000156459019038256/0001564590-19-038256.txt', '0001564590-19-026445': 'https://www.sec.gov/Archives/edgar/data/0001318605/000156459019026445/0001564590-19-026445.txt', '0001564590-19-013462': 'https://www.sec.gov/Archives/edgar/data/0001318605/000156459019013462/0001564590-19-013462.txt', '0001564590-19-003165': 'https://www.sec.gov/Archives/edgar/data/0001318605/000156459019003165/0001564590-19-003165.txt', '0001564590-18-026353': 'https://www.sec.gov/Archives/edgar/data/0001318605/000156459018026353/0001564590-18-026353.txt

In [5]:
AN_Active = {}

# Get filename, method 3, use htm.# anchor to jump to Balnce sheet.
for AN, url in Accession_Number_URL.items():
    
    year = int( AN.split( sep='-' )[1] )
    
    # Setting limits.
    lower_limit = 8
    upper_limit = 21
    if lower_limit < year < upper_limit:
        print('~'*75)
        print('url - ', url)
        
        r = requests.get(url)

        try: 
            soup = BeautifulSoup(r.content, "html")

        except:
            print('Look at: ')
            print( '{} - {}'.format(AN, url))

        filing_document = soup.find('document')
        document_id = filing_document.type.find(text=True, recursive=False).strip()
        print('document_id - ', document_id)

        if document_id == '10-Q' or document_id == '10-K':

            # 1.0
            AccessionNumber = AN.replace('-', '')

            # 2.1 part of table website.
            filename = str(soup.find('filename').contents[0]).replace('\n', '')
            print('filename - ', filename, '\n')

            # 2.2 Anchor page_key.
            hr_tag = soup.findAll('a', href=True)
            page_key = str( [table.get('href') for table in hr_tag if 'Balance' in table.get_text()][0])
            print('page_key - ', page_key, '\n')

            KEY = filename + page_key

            active_url = new_doc_url.format( xx = AccessionNumber, yy = KEY ).replace('.txt', '')

    #         print('active_url - ', active_url)

            # Key for 2016 onwards is the same, will overwrite the keys() in AN_Active. 
            AN_Active.update({ AN : [ page_key , url ] } )

        else:
            continue

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
url -  https://www.sec.gov/Archives/edgar/data/0001318605/000156459020019931/0001564590-20-019931.txt
document_id -  10-Q
filename -  tsla-10q_20200331.htm 

page_key -  #Consolidated_Balance_Sheets 

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
url -  https://www.sec.gov/Archives/edgar/data/0001318605/000156459020018984/0001564590-20-018984.txt
document_id -  10-K/A
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
url -  https://www.sec.gov/Archives/edgar/data/0001318605/000156459020004475/0001564590-20-004475.txt
document_id -  10-K
filename -  tsla-10k_20191231.htm 

page_key -  #Consolidated_Balance_Sheets 

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
url -  https://www.sec.gov/Archives/edgar/data/0001318605/000156459019038256/0001564590-19-038256.txt
document_id -  10-Q
filename -  tsla-10q_20190930.htm 

page_key -  

document_id -  10-K
filename -  d452995d10k.htm 

page_key -  #tx452995_102 

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
url -  https://www.sec.gov/Archives/edgar/data/0001318605/000119312512457610/0001193125-12-457610.txt
document_id -  10-Q
filename -  d410318d10q.htm 

page_key -  #tx410318_3 

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
url -  https://www.sec.gov/Archives/edgar/data/0001318605/000119312512332138/0001193125-12-332138.txt
document_id -  10-Q
filename -  d364775d10q.htm 

page_key -  #tx364775_3 

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
url -  https://www.sec.gov/Archives/edgar/data/0001318605/000119312512225825/0001193125-12-225825.txt
document_id -  10-Q
filename -  d325967d10q.htm 

page_key -  #tx325967_3 

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
url -  https://www.sec.gov/Archives/edgar/data/0001318605/000119312512137560/0001193

In [7]:
remaining_files_num  = len(Accession_Number_URL) - len(AN_Active)
print(f'number of files not caputured: {remaining_files_num}\n')

today = datetime.datetime.today()
next_year = int(today.strftime('%y'))+1

problem_files = list((set([AN for AN in Accession_Number_URL.keys() if  10 < int(AN.split(sep= '-')[1]) < next_year]) - set(AN_Active.keys()) ))
print('problem_files: \n', problem_files,'\n')

number of files not caputured: 2

problem_files: 
 ['0001193125-12-137560', '0001564590-20-018984'] 



## Cleaning Column Names 

In [8]:
month_dict = {'December' : 'Q4',
              'November' : 'Q4',
             'October' : 'Q4',
             'September' : 'Q3',
             'August': 'Q3',
             'July' : 'Q3',
             'June' : 'Q2',
             'May' : 'Q2',
             'April' : 'Q2',
             'March' : 'Q1',
             'Feburary' : 'Q1',
             'January' : 'Q1'}

In [9]:
col_name_variants = {
    'Inventor' : 'Inventory',
    'Total liabilities and' : 'Total liabilities and equity',
    'Common stock.*' : 'Common Stock',
    'Preferred stock.*': 'Preferred Stock',
    'Accounts receivable.*' : 'Accounts Receivable',
    'Accrued liabilities.*' : 'Accrued Liabilities',
    'Deferred revenue.*' : 'Deferred Revenue',
    'Redeemable noncontrolling interests in subsidiaries.*' : 'Noncontrolling interests in subsidiaries',
    'Short term investments.*' : 'Short term investments', 
    'Property|equipment.' : 'Property and Equipment',
    'Retained earnings.*' : 'Retained Earnings',
    'stockholders.*' : 'Shareholder Equity',
    '^Cash.*' : 'Cash',
    'Convertible senior notes.*' : 'Convertible Senior Notes'
}

In [10]:
def clean_col_name( name ):
    
    for detect, replacement in col_name_variants.items():

        if re.findall(detect, name, flags=re.I):

            n_column = name.replace(name, replacement) 

            return n_column 
            
    return name

## Cleaning Data in Table

In [11]:
# Throw column list-like object into cleaning function. 
def cleaning_column(r_column, m):
     
    column_list = []
    column_name = []
    
    for element_post, element in enumerate(r_column):
        
        # Handling the first "row" in the table i.e. column name.
        if element_post == 0:
#             print(element)
            ele_norm = unicodedata.normalize('NFKD', element)
            
            ele_norm = ele_norm.replace('net', '')
            
            ele_norm = ele_norm.replace( '-', ' ')
            
            s_ele_norm = re.sub(r"^\s+|\s+$", "", ele_norm)
            
            name = re.sub(r'(?=,|:|\.|\(){1}.*', repl='', string= s_ele_norm, flags=re.I)
            
            # clean_col_name function Choose old_if or new_dict
            clean_name = clean_col_name( name )
            ###
            column_name.append(clean_name)
            ###
            column_list.append(clean_name)
        
        # Handling the all the remaining "rows" in the table i.e. data/numbers.
        elif element not in ['\n$', '\n\xa0', '\n']:
                        
            n_element = element.replace( ',', '' )

            pattern = re.compile(r'[\d]+,[\d]+|[\d]+')

            res = pattern.findall(n_element)

            res = [int(ele) * m for ele in res]

            column_list.extend(res)

    return column_list
    

In [12]:
# New and imporved.
def Quarter( column ):
    
#     print('column used for quarter', column)
    
    multi_list = [element.split() for element in column if element.split() != []]
#     print('multi_list', multi_list)
    new_column = [item for sublist in multi_list for item in sublist]
#     print('new_column used for year', new_column)
    
    Q = []
    Y = []
    
    for letter in column:
#         print('letter', letter)
        try:
            month = re.findall(r'\w{}+'.format(list(month_dict.keys())), letter.replace(' ',  '') )[0]
            
            Q_num = month_dict[month] 
    #         print('Q_num' , Q_num)
    
            if Q_num != []:
                Q.append(Q_num)
                
        except:
            continue
    #         print('~'*20)
        
#     print('Q' , Q) 
#     Q_final = Q[0] 
#     print('Q_final' , Q_final)
    
    Q_final =['Quarter'] + Q[:1]
    
    
    for letter in new_column:   
        
        #print( letter.split(',') )
        
        Year = [re.compile(r'^[0-9]{4}$').findall(p_s_ele_norm) for p_s_ele_norm in letter.split(',') if re.compile(r'^[0-9]{4}$').findall(p_s_ele_norm) != []]
        Y.append(Year)
#     print(f'Y - {Y}')
    Y = [item for sublist in Y for item in sublist][0]
    
    Y.insert(0, 'Year')
    #Y.insert(1, 0)
    #Y.insert(1, 0)
    return Q_final, Y
    print( '\n', 'Q_final', Q_final, 'Y', Y)

# Recursive solution to finding tables and specific text in HTML.

## Looking for "millions" or "thousands", ie multiplier

In [77]:
# Multiplier finder.
def finding_m(current_sibling, nex_sibling=0):
    
    global multiplier
#     print(f'current_sibling - {current_sibling}')
    
    # Base Case
    if search(r'in thousands,', current_sibling.parent.text, flags=re.I):
        multiplier = 10**3
#         print(f'current_sibling - {current_sibling.text}')
        return multiplier
    
    if search(r'in millions,', current_sibling.parent.text, flags=re.I):
        multiplier = 10**6
#         print(f'current_sibling - {current_sibling.text}')
        return multiplier

    
    else:
        return finding_m(nex_sibling, nex_sibling.next_sibling)


In [62]:
print(search(r'Million', page_key.text, flags=re.I)==True)

False


In [78]:
finding_m(table)

1000000

In [14]:
table = None
def getting_m(page, n):
    
#     print('page{}'.format('.parent'*n))

    #Setting base case. 
    recursive_limit = 5
    
    while n<recursive_limit:
        
        try: #try the current tag level.
                
            return finding_m(page, page.next_sibling)
        
        except AttributeError: # Handling None type object has no 'next_sibling' exception. 

#             print('\n --- Trying the next parent --- \n')
            n += 1
            getting_m(page.parent, n) # try the same function again, this time use the parent-tag.

## Conceptually: Returns Immediate Table after "key-tag" anchor. 

In [15]:
# Working next_sibling finder.
def next_content(current_sibling, next_cont):
    
    # Base Case
    if (current_sibling == 'table'): #or current_sibling.name =='div'):
        table = current_sibling
#         print(f'inner current_sibling - {current_sibling.text}')
        return current_sibling
    
    else:
        return next_content(current_sibling, current_sibling.contents[0])


In [52]:
# Working next_sibling finder.
def nextS(current_sibling, nex_sibling):
    
    global table
#     print(f'nextSOutercurrent_sibling - {current_sibling}')
    
    # Base Case
    if (current_sibling.name == 'table') or (current_sibling.name =='div'):
        table = current_sibling
#         print(f'nextSInnercurrent_sibling - {current_sibling.text}')
        return current_sibling
        
    else:
        return nextS(nex_sibling, nex_sibling.next_sibling)


In [53]:
table = None
def Table_Creator(page, n):
    
#     print('page{}'.format('.parent'*n))
    
    while n<3:
        
        try: #try the current tag level.
            nextS(page, page.next_sibling)
            return table 
        
        except AttributeError: # Handling None type object has no 'next_sibling' exception. 

#             print('\n --- Trying the next parent --- \n')
            n += 1
            Table_Creator(page.parent, n) # try the same function again, this time use the parent-tag.

# Actual Executed Code 

In [80]:
Checklist ={}

for AN, key_url in AN_Active.items():
    
    year = int( AN.split( sep='-' )[1] )
    print('~'*75)
    # Setting limits.
    lower_limit = 10
    upper_limit = 21
    if lower_limit < year < upper_limit:
    
        print(f'Accession Number - {AN}\n')
        key = key_url[0].replace('#', '')
        print(f'key - {key}\n')
        act_url = key_url[1]
        print(f'Active URL - {key_url[1]}\n')

        r = requests.get(act_url)

        # Not choosing r.encoding to easier modify 
    #         r.encoding = 'utf-8'

        soup = BeautifulSoup(r.content, "lxml")

        filing_document = soup.find('document')
        document_id = filing_document.type.find(text=True, recursive=False).strip()
        print(f'document_id - {document_id}\n' )

        table = None
        multiplier = None
    # *********************
        page_key = soup.find(attrs={"name": key} )

        if (page_key != None):
            print(f'First try - {page_key}')

        else: 
            page_key = soup.find(attrs={"id": key} )
            print(f'page_key id - {page_key}\n' )

        if (page_key == None):          
            print(f'page_key is missing in the document.\n Checklist is updated. \n' )
            Checklist.update({document_id: AN})
            continue        
    # ********************        

        getting_m(page_key, n=0)
        
        Table_Creator(page_key, n=0)
#         print(table)
        if multiplier == None:
            finding_m(table)
#         table = page_key.next_sibling.next_sibling.next_sibling.next_sibling.contents[0]
        target = table.findAll('tr')
#         print(f'target - {target[0]}\n')

        row_list = [ row for row in target if row.get_text(strip=True, separator=',') != '']
#         print(f'row_list - {row_list[0]}\n')

        col_list = []

        for row in row_list:
            col = row.find_all('td')
    #             print(f'col - {col}')
            column = []
            for tr in col:
    #                 print(f'tr - {tr}')
                ele = tr.text
#                 print(f'ele - {ele}')
                try:
                    ele= ele.encode('cp1252').decode("utf-8", "ignore")
    #                     print(f'cp1252 encode')
                except:
                    ele= ele.encode('latin1').decode("utf-8", "ignore")
    #                     print(f'latin1 encode')
                column.append(ele)
#                 print(f'column - {column}')
            col_list.append(column)

    #         col_list.pop(0)

        new_col_list = []

#         print(col_list)
        # Tesla Date is split to Q[0] and Y[1].
        col_list[0] = col_list[0] + col_list[1]
        col_list.pop(1)

        for col_num, column in enumerate(col_list): 

            if col_num == 0:
    #                 print('raw column - ', column)
                Q, Y = Quarter(column)
                print(Q, Y)
                new_col_list.append(Q)
                new_col_list.append(Y)

            else:
                column.pop(1)
                column.pop(1)
    #                 print(f'column - {column}')
                new_col_list.append(cleaning_column( column, multiplier ))

        df = pd.DataFrame(new_col_list)
        df = df.T.reset_index(drop=True)
        df.columns = df.iloc[0]
        df = df.drop([row for row in range(df.shape[0]) if row != 1], axis=0).reset_index(drop=True)
        ICD.display(df)

        with open('{}-{}.csv'.format(AN, document_id), "wb") as fp:   # Pickling
            pickle.dump(df, fp)

        # display a status to the user.
        print('All the pages from {} document {} have been tableized and saved.'.format(AN, document_id))    
        print('-'*80) 

if Checklist == {}:
    print('\nEverything went well.\n')

else: 
    print('\n******************These files have a problem.******************\n', Checklist)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Accession Number - 0001564590-20-019931

key - Consolidated_Balance_Sheets

Active URL - https://www.sec.gov/Archives/edgar/data/0001318605/000156459020019931/0001564590-20-019931.txt

document_id - 10-Q

page_key id - <p id="Consolidated_Balance_Sheets" style="text-align:center;margin-bottom:0pt;margin-top:4pt;text-indent:0%;font-weight:bold;font-family:Times New Roman;font-size:10pt;font-style:normal;text-transform:none;font-variant: normal;">Consolidated Balance Sheets</p>

['Quarter', 'Q1'] ['Year', '2020']


,Quarter,Year,Assets,Current assets,Cash,Accounts Receivable,Inventory,Prepaid expenses and other current assets,Total current assets,Operating lease vehicles,Solar energy systems,Property and Equipment,Operating lease right of use assets,Intangible assets,Goodwill,Other non current assets,Total assets,Liabilities,Current liabilities,Accounts payable,Accrued Liabilities,Deferred Revenue,Customer deposits,Current portion of debt and finance leases,Total current liabilities,Debt and finance leases,Deferred Revenue,Other long term liabilities,Total liabilities,Commitments and contingencies,Noncontrolling interests in subsidiaries,Convertible Senior Notes,Equity,Shareholder Equity,Preferred Stock,Common Stock,Additional paid in capital,Accumulated other comprehensive loss,Accumulated deficit,Shareholder Equity,Noncontrolling interests in subsidiaries,Total liabilities and equity
0,Q1,2020,None,None,8080000000,1274000000,4494000000,1045000000,14893000000,2527000000,6106000000,10638000000,1197000000,323000000,193000000,1373000000,37250000000,None,None,3970000000,2825000000,1186000000,788000000,3217000000,11986000000,10666000000,1199000000,2667000000,26518000000,None,632000000,60000000,None,None,None,0,15390000000,113000000,6104000000,9173000000,867000000,37250000000


All the pages from 0001564590-20-019931 document 10-Q have been tableized and saved.
--------------------------------------------------------------------------------
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Accession Number - 0001564590-20-004475

key - Consolidated_Balance_Sheets

Active URL - https://www.sec.gov/Archives/edgar/data/0001318605/000156459020004475/0001564590-20-004475.txt

document_id - 10-K

page_key id - <p id="Consolidated_Balance_Sheets" style="text-align:center;margin-bottom:0pt;margin-top:4pt;text-indent:0%;font-weight:bold;font-family:Times New Roman;font-size:10pt;font-style:normal;text-transform:none;font-variant: normal;">Consolidated Balance Sheets</p>

['Quarter', 'Q4'] ['Year', '2019']


,Quarter,Year,Assets,Current assets,Cash,Restricted cash,Accounts Receivable,Inventory,Prepaid expenses and other current assets,Total current assets,Operating lease vehicles,Solar energy systems,Property and Equipment,Operating lease right of use assets,Intangible assets,Goodwill,MyPower customer notes receivable,Restricted cash,Other assets,Total assets,Liabilities,Current liabilities,Accounts payable,Accrued Liabilities,Deferred Revenue,Resale value guarantees,Customer deposits,Current portion of debt and finance leases,Total current liabilities,Debt and finance leases,Deferred Revenue,Resale value guarantees,Other long term liabilities,Total liabilities,Commitments and contingencies,Noncontrolling interests in subsidiaries,Equity,Shareholder Equity,Preferred Stock,Common Stock,Additional paid in capital,Accumulated other comprehensive loss,Accumulated deficit,Shareholder Equity,Noncontrolling interests in subsidiaries,Total liabilities and equity
0,Q4,2019,None,None,6268000000,246000000,1324000000,3552000000,713000000,12103000000,2447000000,6138000000,10396000000,1218000000,339000000,198000000,393000000,269000000,808000000,34309000000,None,None,3771000000,2905000000,1163000000,317000000,726000000,1785000000,10667000000,11634000000,1207000000,36000000,2655000000,26199000000,None,643000000,None,None,None,0,12737000000,36000000,6083000000,6618000000,849000000,34309000000


All the pages from 0001564590-20-004475 document 10-K have been tableized and saved.
--------------------------------------------------------------------------------
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Accession Number - 0001564590-19-038256

key - CONSOLIDATED_BALANCE_SHEETS

Active URL - https://www.sec.gov/Archives/edgar/data/0001318605/000156459019038256/0001564590-19-038256.txt

document_id - 10-Q

page_key id - <p id="CONSOLIDATED_BALANCE_SHEETS" style="text-align:center;margin-bottom:0pt;margin-top:4pt;text-indent:0%;font-weight:bold;font-family:Times New Roman;font-size:10pt;font-style:normal;text-transform:none;font-variant: normal;">Consolidated Balance Sheets</p>

['Quarter', 'Q3'] ['Year', '2019']


...

,Quarter,Year,Assets,Current assets,Cash,Restricted cash and marketable securities,Accounts Receivable,Inventory,Prepaid expenses and other current assets,Total current assets,Operating lease vehicles,Property and Equipment,Restricted cash,Other assets,Total assets,Shareholder Equity,Current liabilities,Accounts payable,Accrued Liabilities,Deferred Revenue,Capital lease obligations,Customer deposits,Convertible Senior Notes,Total current liabilities,Capital lease obligations,Deferred Revenue,Convertible Senior Notes,Resale value guarantee,Other long term liabilities,Total liabilities,Commitments and contingencies,Convertible Senior Notes,Shareholder Equity,Preferred Stock,Common Stock,Additional paid in capital,Accumulated deficit,Shareholder Equity,Total liabilities and equity
0,Q2,2014,None,None,2674910000,11714000,96607000,596927000,61536000,3441694000,531226000,1035822000,7247000,38474000,5054463000,None,None,443548000,161437000,139436000,9700000,228017000,593712000,1575850000,14099000,234980000,1767024000,345192000,98902000,4036047000,None,66083000,None,None,125000,2203535000,1251327000,952333000,5054463000


All the pages from 0001193125-14-303175 document 10-Q have been tableized and saved.
--------------------------------------------------------------------------------
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Accession Number - 0001193125-14-192606

key - tx715897_2

Active URL - https://www.sec.gov/Archives/edgar/data/0001318605/000119312514192606/0001193125-14-192606.txt

document_id - 10-Q

First try - <a name="tx715897_2"></a>
['Quarter', 'Q1'] ['Year', '2014']


,Quarter,Year,Current assets,Cash,Short term marketable securities,Restricted cash,Accounts Receivable,Inventory,Prepaid expenses and other current assets,Total current assets,Operating lease vehicles,Property and Equipment,Restricted cash,Other assets,Total assets,Shareholder Equity,Current liabilities,Accounts payable,Accrued Liabilities,Deferred Revenue,Capital lease obligations,Customer deposits,Convertible debt,Total current liabilities,Capital lease obligations,Deferred Revenue,Long term debt,Resale value guarantee,Other long term liabilities,Total liabilities,Commitments and contingencies,1,Shareholder Equity,Preferred Stock,Common Stock,Additional paid in capital,Accumulated deficit,Shareholder Equity,Total liabilities and equity
0,Q1,2014,None,2393908000,189111000,1049000,72380000,450730000,48869000,3156047000,451729000,849389000,7102000,36143000,4500410000,None,None,375778000,128674000,112740000,8397000,198006000,589875000,1413470000,12572000,210817000,1519967000,290617000,70969000,3518412000,None,69942000,None,None,124000,2101352000,1189420000,912056000,4500410000


All the pages from 0001193125-14-192606 document 10-Q have been tableized and saved.
--------------------------------------------------------------------------------
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Accession Number - 0001193125-14-069681

key - tx668062_26

Active URL - https://www.sec.gov/Archives/edgar/data/0001318605/000119312514069681/0001193125-14-069681.txt

document_id - 10-K

First try - <a name="tx668062_26"></a>


AttributeError: 'NoneType' object has no attribute 'findAll'

In [67]:
multiplier

1000

In [47]:
page_key.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.name

'div'

In [31]:
table = page_key.parent.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.contents[0].contents[0]
print(table)

AttributeError: 'NoneType' object has no attribute 'next_sibling'

In [ ]:
target = table.findAll('tr')

In [36]:
table

<p id="BALANCE_SHEETS" style="text-align:center;margin-bottom:0pt;margin-top:0pt;text-indent:0%;font-weight:bold;font-size:10pt;font-family:Arial;font-style:normal;text-transform:none;font-variant: normal;">BALANCE SHEETS </p>

In [ ]:
from IPython.display import display_html

table_rows = str(soup.find('table')) # convert soup back to html
display_html(table_rows, raw=True) 

df = pd.read_html(table_rows)[0] # Read the first element in read_html list 
ICD.display(df.T)

In [22]:
QorK = {'10-Q': ['0001564590-20-019706',
  '0001564590-20-002450',
  '0001564590-19-037549',
  '0001564590-19-012709',
  '0001564590-19-001392',
  '0001564590-18-024893',
  '0001564590-18-009307',
  '0001564590-18-001129',
  '0001564590-17-020171',
  '0001564590-17-007547',
  '0001564590-17-000654',
  '0001193125-16-742796',
  '0001193125-16-550254',
  '0001193125-16-441821',
  '0001193125-15-350718',
  '0001193125-15-144151',
  '0001193125-15-020351',
  '0001193125-14-380252',
  '0001193125-14-157088',
  '0001193125-14-018634',
  '0001193125-13-409855',
  '0001193125-13-160748',
  '0001193125-13-022207',
  '0001193125-12-427209'],
 '10-K': ['0001564590-19-027952',
  '0001564590-18-019062',
  '0001564590-17-014900',
  '0001193125-16-662209',
  '0001193125-15-272806',
  '0001193125-14-289961',
  '0001193125-13-310206',
  '0001193125-12-316848']}
